# Type Level Programming

- Higher Kinded Polymorphism
- Vectors with type level length
- Heterogenous lists

### Higher kinded polymorphism

In [6]:
data HigherKinded f a
  = Bare a
  | Wrapped (f a)
  
:k HigherKinded

let wrapped = Wrapped $ Just 4
:t wrapped

HigherKinded :: (* -> *) -> * -> *

wrapped :: forall a. Num a => HigherKinded Maybe a

### Length Indexed Vectors
The `DataKinds` extension allows us to promote a value constructor to a type constructor and the type constructor to a kind constructor. We prefix the constructor with `'` to designate a promotion.
Let's define a peano number system that we will use on the type level

In [17]:
:ext DataKinds
:ext GADTs

data Nat = Zero | Succ Nat

:k 'Zero
:k 'Succ

:t Zero
:t Succ

'Zero :: Nat

'Succ :: Nat -> Nat

Zero :: Nat

Succ :: Nat -> Nat

In [24]:
:ext KindSignatures

data Vector (n :: Nat) a where -- ghc can infer the :: Nat if we want
  VNil :: Vector 'Zero a
  VCons :: a -> Vector n a -> Vector ('Succ n) a
  
instance Show a => Show (Vector n a) where
  show VNil = "VNil"
  show (VCons x xs) = "VCons " ++ show x ++ " (" ++ show xs ++ ")"
  
let v = VCons 4 $ VCons 3 $ VCons 2 VNil
:t v
show v

v :: forall a. Num a => Vector ('Succ ('Succ ('Succ 'Zero))) a

"VCons 4 (VCons 3 (VCons 2 (VNil)))"

Now we want to define an `append` combinator to append a vector on to the end of another vector.
First we need a way to add two `Num`s together. Type level functions are call `TypeFamilies`.

In [37]:
:ext TypeFamilies

type family Add n m where
  Add 'Zero n = n
  Add ('Succ n) m = 'Succ (Add n m)

append :: Vector n a -> Vector m a -> Vector (Add n m) a -- using the Add type family
append VNil xs = xs
append (VCons a as) b = VCons a $ append as b

let v2 = VCons 5 $ VCons 6 $ VCons 7 VNil
let cv = append v v2
:t cv
show cv

cv :: forall a. Num a => Vector ('Succ ('Succ ('Succ ('Succ ('Succ ('Succ 'Zero)))))) a

"VCons 4 (VCons 3 (VCons 2 (VCons 5 (VCons 6 (VCons 7 (VNil))))))"

### Heterogenous Lists
Let's make heterogenous lists (lists containing items of various types) by having attaching a type level list of the types that the heterogenous lists contains to it's type signature.
To use ordinary lists on the type level, we need to `TypeOperators` extension

In [41]:
:ext TypeOperators

data HList xs where
  HNil :: HList '[]
  HCons :: x -> HList xs -> HList (x ': xs)
  
let h = HCons "hello" $ HCons 4 $ HCons (Just 3) HNil
:t h

h :: forall a x. (Num x, Num a) => HList '[[Char], x, Maybe a]

Now let's try writing an instance of `Show` for `HList`
We will write inductive type class instances to do this. It requires the `FlexibleInstances` extension

In [53]:
:ext FlexibleInstances
:ext FlexibleContexts

instance Show (HList '[]) where
  show HNil = "HNil"

instance (Show (HList xs), Show x) => Show (HList (x ': xs)) where
  show (HCons x xs) = "HCons " ++ show x ++ " (" ++ show xs ++ ")"
  
show h

"HCons \"hello\" (HCons 4 (HCons Just 3 (HNil)))"